In [10]:
import json
import requests

In [11]:
def extract():
    f = open('tmdb.json')
    if f:
        return json.loads(f.read());

In [12]:
movieDict = extract()
analysis = {
    "analyser": {
        "default": {
            "type": "english"
        }
    }
}

In [13]:
def reindex(analysisSettings={}, mappingSettings={}, movieDict={}):
    settings = {
        "number_of_ shards": 1,
        "index": {
            "analysis": analysisSettings,
        }
    }
    if mappingSettings:
        settings['mappings'] = mappingSettings
        
    resp = requests.delete('http://192.168.32.10:9200/tmdb')
    resp = requests.put('http://192.168.32.10:9200/tmdb', data=json.dumps(settings))
    bulkMovies = ""
    for id, movie in movieDict.iteritems():
        addCmd = {"index": {"_index": "tmdb",
                           "_type": "movie",
                           "_id": movie["id"]}}
        bulkMovies += json.dumps(addCmd) + "\n" + json.dumps(movie) + "\n"
    resp = requests.post("http://192.168.32.10:9200/_bulk", data=bulkMovies)

In [ ]:
reindex(analysisSettings=analysis, mappingSettings=None, movieDict=movieDict)

In [6]:
usersSearch = 'basketball with cartoon aliens'
query = {
    "size": "20",
    "query": {
        "multi_match": {
            "query": usersSearch,
            "fields": ["title^0.1", "overview"],
        }
    }
}

In [8]:
def search(query):
    url = 'http://192.168.32.10:9200/tmdb/movie/_search'
    httpResp = requests.get(url, data=json.dumps(query))
    searchHits = json.loads(httpResp.text)['hits']
    print "Num\tRelevance Score \t\tMovie Title"
    for idx, hit in enumerate(searchHits['hits']):
        print "%s\t%s\t\t%s" % (idx + 1, hit['_score'], hit['_source']['title'])

In [9]:
search(query)

Num	Relevance Score 		Movie Title
1	0.40751222		Meet Dave
2	0.27429563		Semi-Pro
3	0.26320052		Speed Racer
4	0.25425476		Aliens in the Attic
5	0.23608768		The Watch
6	0.23464145		Alien: Resurrection
7	0.23318566		Bedazzled
8	0.21495439		Space Jam
9	0.20845622		District 9
10	0.20734486		Teen Wolf
11	0.18881755		They Live
12	0.18289658		Grown Ups
13	0.17233244		Men in Black 3
14	0.12361832		White Men Can't Jump
15	0.12361832		Coach Carter
16	0.122831084		The Flintstones
17	0.11145923		Galaxy Quest
18	0.10765152		Pitch Black
19	0.098894656		Batteries Not Included
20	0.09270038		High School Musical
